## TVB TimeSeries Widget

#### This notebook is dedicated to showcasing the TimeSeries widget using TVB data

---

### Intial setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

### Imports

In [3]:
from tvbwidgets.api import TimeSeriesWidget
from tvbwidgets.api import TSWidgetBuilderFromTVB, TSWidgetFactory
import numpy as np
import mne
from IPython.core.display_functions import display

30-03-2022 01:49:13 - DEBUG - tvbwidgets - Package is not fully installed
30-03-2022 01:49:13 - DEBUG - tvbwidgets - Version read from the internal _version.py file
30-03-2022 01:49:13 - INFO - tvbwidgets - Version: 0.1.dev25+g9d7cb39.d20220222


C:\ProgramData\Anaconda3\envs\tvb-widgets\lib\site-packages\pkg_resources\__init__.py:122: PkgResourcesDeprecationWarning: -PKG-VERSION is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
# imports for generating data
from tvb.simulator.lab import *
from tvb.datatypes import time_series
from tvb.basic.config import settings
import tvb.datatypes.time_series

### Go to Generate TVB Data 2

### Generate TVB data

##### (But this has only 1 state variable and 1 mode)

In [5]:
conn = connectivity.Connectivity.from_file()

# configure stimulus spatial pattern
weighting = np.zeros((76, ))
weighting[[14, 52, 11, 49]] = 0.1

WARNING  File 'hemispheres' not found in ZIP.


In [6]:
eqn_t = equations.PulseTrain()
eqn_t.parameters['onset'] = 1.5e3
eqn_t.parameters['T'] = 100.0
eqn_t.parameters['tau'] = 50.0

In [7]:
stimulus = patterns.StimuliRegion(
    temporal=eqn_t,
    connectivity=conn,
    weight=weighting)

In [8]:
#Configure space and time
stimulus.configure_space()
stimulus.configure_time(np.arange(0., 3e3, 2**-4))

In [9]:
sim = simulator.Simulator(
    model=models.Generic2dOscillator(a=np.array([0.3]), tau=np.array([2])),
    connectivity=conn,
    coupling=coupling.Difference(a=np.array([7e-4])),
    integrator=integrators.HeunStochastic(dt=0.5, noise=noise.Additive(nsig=np.array([5e-5]))),
    monitors=(
        monitors.TemporalAverage(period=1.0),
        ),
    stimulus=stimulus,
    simulation_length=5e3, # 1 minute simulation
).configure()

(tavg_time, tavg_data),  = sim.run()

   INFO  White noise configured with dt=0.5


In [10]:
tsr = tvb.datatypes.time_series.TimeSeriesRegion(
    data=tavg_data,
    connectivity=conn,
    sample_period=sim.monitors[0].period / 1000.0, 
    sample_period_unit="s")
tsr.configure()
tsr

,value
Dimensions,"('Time', 'State Variable', 'Region', 'Mode')"
Length,5.0
Region Mapping,None
Region Mapping Volume,None
Sample period,0.001
Source Connectivity,Connectivity gid: 16cff496-65c9-49b1-ba3d-e0a9d90ccfc0
Time units,s
Time-series name,TimeSeriesRegion gid: 9f38dd36-f8f5-48bd-972f-f1b34d4a5adf
Time-series type,TimeSeriesRegion
"[min, median, max]","[-1.53398, 0.0317675, 3.65151]"


### Generate TVB data 2

In [5]:
jrm = models.JansenRit(mu=np.array([0.]), v0=np.array([6.]))
monitor = monitors.TemporalAverage(period=2 ** -2)

# the other aspects of the simulator are standard
sim = simulator.Simulator(
    model=jrm,
    connectivity=connectivity.Connectivity.from_file(),
    coupling=coupling.SigmoidalJansenRit(a=np.array([10.0])),
    monitors=(monitor,),
    simulation_length=1e3,
).configure()

# run it
(time_array, data_array), = sim.run()

WARNING  File 'hemispheres' not found in ZIP.
WARNING  random_state supplied for non-stochastic integration


C:\ProgramData\Anaconda3\envs\tvb-widgets\lib\site-packages\tvb\simulator\coupling.py:375: RuntimeWarning: overflow encountered in exp
  pre = self.cmax / (1.0 + numpy.exp(self.r * (self.midpoint - (x_j[:, 0] - x_j[:, 1]))))
C:\ProgramData\Anaconda3\envs\tvb-widgets\lib\site-packages\numba\np\ufunc\gufunc.py:151: RuntimeWarning: overflow encountered in _numba_dfun_jr
  return self.ufunc(*args, **kwargs)


In [6]:
type(data_array)

numpy.ndarray

In [7]:
data_array.shape

(4096, 4, 76, 1)

In [8]:
tsr = time_series.TimeSeries(data=data_array, time=time_array, sample_period=monitor.period/1000, sample_period_unit="s")
tsr

,value
Dimensions,"('Time', 'State Variable', 'Space', 'Mode')"
Length,1.024
Sample period,0.00025
Time units,s
Time-series name,TimeSeries gid: 220d3fbb-ca17-4ca5-8c59-4891495bf6f0
Time-series type,TimeSeries
"[min, median, max]","[-3588.33, 0.162523, 3692.42]"
dtype,float64
shape,"(4096, 4, 76, 1)"


In [9]:
tsr.sample_rate

4000.0

### Plot data using widget

In [11]:
director = TSWidgetFactory(data = tsr)

In [12]:
ts = director.create_ts_widget()

Creating RawArray with float64 data, n_channels=76, n_times=4096
    Range : 0 ... 4095 =      0.000 ...     1.024 secs
Ready.


In [13]:
display(ts.get_widget())

Using matplotlib as 2D backend.


GridBox(children=(Accordion(children=(VBox(children=(HBox(children=(VBox(children=(Checkbox(value=True, descri…